# The Determinants of Turnout

In [1]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      StataNow 19.5
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2025 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-782-8272        https://www.stata.com
                                   979-696-4600        service@stata.com

Stata license: Unlimited-user 2-core network, expiring  9 Sep 2026
Serial number: 501909305069
  Licensed to: 罗奕辰
               UCL

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


In [2]:
%%stata

clear all
set more off
set varabbrev off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"


. 
. clear all

. set more off

. set varabbrev off

. version 18

. 
. * Paths
. global PROCESSED_DATA "processed_data"

. global TABLES "tables"

. 


In [3]:
%%stata

import delimited using "$PROCESSED_DATA/proposals_panel.csv", clear

* Parse date
gen day = date(date, "YMD")
format day %td
gen month = month(day)
gen quarter = quarter(day)

gen year = year(day)
encode gecko_id, gen(token)
encode space,    gen(dao)

* Dependent variables
local dep non_whale_participation whale_participation

* Independent variables
local indep weighted ranked_choice multi_choices 
local discussion concensus professionalism objectiveness data_intensity hhi_post_number

* Label variables
label var weighted "\${\it Weighted}_{i,t}\$"
label var quadratic "\${\it Quadratic Voting}_{i,t}\$"
label var ranked_choice "\${\it Ranked Choice}_{i,t}\$"
label var multi_choices "\${\it Multiple Choices}_{i,t}\$"
label var professionalism  "\${\it Professionalism}_{i,t}\$"
label var objectiveness    "\${\it Objectiveness}_{i,t}\$"
label var concensus       "\${\it Concensus}_{i,t}\$"
label var data_intensity   "\${\it Data Intensity}_{i,t}\$"
label var hhi_post_number  "\${\it HHI}_{i,t}^{\# of Post}\$"


. 
. import delimited using "$PROCESSED_DATA/proposals_panel.csv", clear
(encoding automatically selected: ISO-8859-1)
(54 vars, 2,841 obs)

. 
. * Parse date
. gen day = date(date, "YMD")

. format day %td

. gen month = month(day)

. gen quarter = quarter(day)

. 
. gen year = year(day)

. encode gecko_id, gen(token)

. encode space,    gen(dao)

. 
. * Dependent variables
. local dep non_whale_participation whale_participation

. 
. * Independent variables
. local indep weighted ranked_choice multi_choices 

. local discussion concensus professionalism objectiveness data_intensity hhi_p
> ost_number

. 
. * Label variables
. label var weighted "\${\it Weighted}_{i,t}\$"

. label var quadratic "\${\it Quadratic Voting}_{i,t}\$"

. label var ranked_choice "\${\it Ranked Choice}_{i,t}\$"

. label var multi_choices "\${\it Multiple Choices}_{i,t}\$"

. label var professionalism  "\${\it Professionalism}_{i,t}\$"

. label var objectiveness    "\${\it Objectiveness}_{i,t}\$"

. label var

## Proposal Characteristics

In [4]:
%%stata

******************************************************
* LOGISTIC REGRESSIONS
******************************************************

eststo clear

foreach d of local dep {

    * VIF test
    qui reg `d' `indep' `discussion'
    estat vif

    * Run regression without discussion
    reghdfe `d' `indep', absorb(year dao)
    eststo `d'
    estadd local fe_token "Y"
    estadd local fe_time  "Y"

    * Run regression with discussion
    reghdfe `d' `indep' `discussion', absorb(year dao)
    eststo `d'_d
    estadd local fe_token "Y"
    estadd local fe_time  "Y"
}

* Export LaTeX table
esttab                                                            ///
    non_whale_participation non_whale_participation_d             ///
    whale_participation whale_participation_d                     ///
    using "$TABLES/reg_participation_char.tex", replace           ///
    se star(* 0.10 ** 0.05 *** 0.01) label nogaps nocon           ///
    nonotes booktabs nomtitles                                    ///
    b(%9.3f) se(%9.2f)                                           ///
    mgroups("\${\it Participation}^{Small}_{i,t}\$"              ///
            "\${\it Participation}^{Whale}_{i,t}\$",             /// 
            span                                                 ///
            pattern(1 0 1 0)                                     ///
            prefix(\multicolumn{@span}{c}{)                      ///
            suffix(})                                            ///
            erepeat(\cmidrule(lr){@span}) )                      ///
    substitute("\_" "_")                                         ///
    stats(fe_token fe_time N r2_a,                               ///
        fmt(0 0 %9.0fc %9.3f)                                   ///
         labels("Token FE" "Year FE" "Observations" "Adjusted R²"))


. 
. ******************************************************
. * LOGISTIC REGRESSIONS
. ******************************************************
. 
. eststo clear

. 
. foreach d of local dep {
  2. 
.     * VIF test
.     qui reg `d' `indep' `discussion'
  3.     estat vif
  4. 
.     * Run regression without discussion
.     reghdfe `d' `indep', absorb(year dao)
  5.     eststo `d'
  6.     estadd local fe_token "Y"
  7.     estadd local fe_time  "Y"
  8. 
.     * Run regression with discussion
.     reghdfe `d' `indep' `discussion', absorb(year dao)
  9.     eststo `d'_d
 10.     estadd local fe_token "Y"
 11.     estadd local fe_time  "Y"
 12. }

    Variable |       VIF       1/VIF  
-------------+----------------------
hhi_post_n~r |      1.78    0.562372
data_inten~y |      1.72    0.582687
objectiven~s |      1.58    0.634225
   concensus |      1.50    0.668429
multi_choi~s |      1.46    0.684871
    weighted |      1.45    0.690011
profession~m |      1.18    0.844243
ranked_c